# Lab 3: Modelos de clasificación en presencia de clases desbalanceadas


## ¡Bienvenido/a!

Te invitamos a realizar el primer trabajo.
- Objetivo: Para un caso práctico desarrollado en Python, aplicar diferentes técnicas de clasificación a un conjunto de datos con clases desbalanceadas, donde también se aplicarán diferentes alternativas para balancear las clases.
- Tipo de actividad: Individual
- Tipo de evaluación: Sumativa 
- Ponderación: 12%
- Puntaje: 100 puntos
- Calificación: Escala de 1 a 7, con una exigencia de 50%. La nota mínima para aprobar es 4.0.


## Introducción:

El objetivo de este laboratorio es explorar el uso de modelos de clasificación en presencia de clases desbalanceadas.
Para esto, vamos a utilizar la base de datos `post_pabellon.csv`, la cual contiene datos de pacientes que fueron
sometidos a una biopsia (extracción de tejido). La variable de interés es `HOSPITALIZACION`, la cual indica si el
paciente fue hospitalizado luego de la biopsia. 

La siguiente tabla contiene la descripción de las variables (buscar información adicional en caso de considerarlo necesario) de la base de datos:

|Variable|Descripción|Tipo|
|--------|-----------|--|
|EDAD|Edad del paciente| Numérica|
|DIABETES|Indica si el paciente tiene o no diabetes| Binaria 1/0|
|HOSPITALIZACIÓN ULTIMO MES|Indica si el paciente fue hospitalizado el mes previo al procedimiento| Binaria 1/0|
|CUP|Uso de cateter urinario al momento de la biopsia| Binaria 1/0|
|ENF. CRONICA PULMONAR OBSTRUCTIVA|Indica si el paciente tiene una efermedad crónica polmunar obstructuva| Binaria 1/0|
|VOLUMEN PROSTATICO|Incica si el volumen prostatico es mayor a 40 $cm^3$| Binaria 1/0|
|PSA| Concentración del antígeno prostático específico en la sangre| Numérica|
|BIOPSIAS PREVIAS| Indica si el paciente ha tenido biopsias previas| Binaria 1/0|
|ANTIBIOTICO UTILIAZADO EN LA PROFILAXIS| Indica el tipo de antibiótico utilizado en la profilaxis| Categórica nominal |
|NUMERO DE MUESTRAS TOMADAS| Número de muestras tomadas en la biopsia| Numérica|
|BIOPSIA| Resultado de la biopsia| Binaria 1/0 (1: maligno, 0: benigno)|
|HOSPITALIZACION| Indica si el paciente fue hospitalizado luego de la biopsia| Binaria 1/0|


Los modelos que vamos a utilizar son los siguientes:
- Support Vector Classifier (SVC)
- Random Forest Classifier (RFC)
- Naive Bayes Classifier (NBC)

Además, vamos a utilizar las siguientes métricas para evaluar el ajuste y también medir el desempeño de los modelos (capacidad predictiva):
- `Accuracy`: Porcentaje de casos correctamente clasificados sobre el total de casos.
- `Precision`: Porcentaje de casos positivos correctamente clasificados sobre el total de casos clasificados como positivos.
- `Recall`: Porcentaje de casos positivos correctamente clasificados sobre el total de casos positivos.
- `F1-Score`: Media armónica entre Precision y Recall.
- `AUC`: Área bajo la curva ROC.

Por último, para equilibrar las clases vamos a utilizar las siguientes técnicas:
- `RandomOverSampler`: Técnica de sobremuestreo que genera muestras sintéticas de la clase minoritaria.
- `RandomUnderSampler`: Técnica de submuestreo que elimina muestras de la clase mayoritaria.
- `NearMiss`: Técnica de submuestreo que elimina muestras de la clase mayoritaria basándonos en la distancia a las muestras de la clase minoritaria.
- `ADASYN`: Técnica de sobremuestreo que genera muestras sintéticas de la clase minoritaria basándonos en la densidad de las muestras de la clase minoritaria.
- Combinación de `RandomOverSampler` y `RandomUnderSampler`.

In [1]:
# Módulos básicos para análisis y manipulación de datos
import numpy as np
import pandas as pd

# Modelos de clasificación
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB

# Módulos para evaluación de modelos
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics

# Módulos para el balanceo de datos
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from imblearn.over_sampling import RandomOverSampler, ADASYN

1. Cargar la base de datos guardada en el archivo `post_pabellon.xlsx`, y guardar la data en el objeto `datos`. Luego, separar las variables de entrada en `X` y la variable respuesta en `y`. En el caso de las variables de entrada crear variables dummies para las variables categoricas nominales de más de dos niveles, para ello utilice la función `get_dummies` de la librería `pandas` con la opción `drop_first=True`.

In [2]:
datos = None      # Variable que debe modificar
y = None          # Variable que debe modificar
X = None          # Variable que debe modificar

# your code here
# cargar dataframe
datos = pd.read_excel('post_pabellon.xlsx', sheet_name='Datos')

# separar variable objetivo
X = datos.drop(columns=['HOSPITALIZACION'], axis=1)
y = datos['HOSPITALIZACION']

# crear variables dummies
X = pd.get_dummies(X, drop_first=True)
X.head()

,EDAD,DIABETES,HOSPITALIZACIÓN ULTIMO MES,PSA,BIOPSIAS PREVIAS,VOLUMEN PROSTATICO,NUMERO DE MUESTRAS TOMADAS,CUP,ENF. CRONICA PULMONAR OBSTRUCTIVA,BIOPSIA,ANTIBIOTICO UTILIAZADO EN LA PROFILAXIS_FLUOROQUINOLONA_AMINOGLICOSIDO,ANTIBIOTICO UTILIAZADO EN LA PROFILAXIS_OROQUINOLONAS,ANTIBIOTICO UTILIAZADO EN LA PROFILAXIS_OTROS
0,53,0,0,4.0,0,1,12,0,0,0,1,0,0
1,56,0,0,7.7,0,1,12,0,0,0,1,0,0
2,53,0,0,7.0,0,1,12,0,0,0,1,0,0
3,65,0,0,4.3,0,0,12,0,0,0,1,0,0
4,62,0,0,7.0,0,1,12,0,0,0,1,0,0


In [3]:
# Test 1 - P1: Prueba de la carga de la base de datos y de la creación de las variables de entrada y respuesta tal como se especifica.

2. Realice una descomposición aleatoria estratificada según la variable respuesta con una proporción de 70% para entrenamiento y 30% para validación. Utilice una semilla de 123 para replicar los resultados.

Debe entregar los siguientes objetos:
- `X_train`: Matriz de predictores para entrenamiento.
- `X_test`: Matriz de predictores para validación.
- `y_train`: Vector de respuesta para entrenamiento.
- `y_test`: Vector de respuesta para validación.

**OBS**: Utilice la función `train_test_split` de la librería `sklearn`.

In [4]:
X_train = None      # Variable que debe modificar
y_train = None      # Variable que debe modificar

X_test = None       # Variable que debe modificar
y_test = None       # Variable que debe modificar

# your code here
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123, stratify=y)

In [5]:
# Test 1 - P2: Prueba de dimensiones de los datos y aleatoriedad de la separación

3. A continuación, debe realizar una **validación cruzada estratificada** con 5 folds sobre la partición de entrenamiento para comparar diferentes modelos que 
se especificarán más adelante. 

Intrucciones:
- Utilice la función `StratifiedKFold` del módulo `model_selection` de la librería `sklearn`. Este objeto debe ser entregado como argumento a la función `cross_val_score`.
- Se debe utilizar el número 123 como semilla de aleatorización de la función `StratifiedKFold`. Además utilice el argumento `shuffle=True`.
- Se deben evaluar las siguientes métricas de ajuste en cada fold: `accuracy`, `precision`, `recall`, `f1` y `roc_auc`.
- El objetivo es comparar el desempeño de los modelos en cada fold, para esto, se debe calcular la **media** de cada métrica de ajuste por modelo.
- Debe entregar un dataframe de nombre `cv_results` con la siguiente estructura:

    | |Accuracy|Precision|Recall|F1|AUC|
    |-|--------|---------|------|--|-------|
    |SV|-|-|-|-|-|
    |RF|-|-|-|-|-|
    |NB|-|-|-|-|-|

    En python:
        
    ```python
    # Dataframe vacío
    cv_results = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])
    ```
- Los modelos deben considerar todas las variables de la base de datos y utilizar los siguientes hiperparámetros:
    - SVC: `C=15`, `kernel='poly'`, `degree=2`
    - RFC: `n_estimators=50`, `max_depth=10`, `random_state=123`, `max_samples=0.8`, `max_features='log2'`
    - NBC: `class_prior=[0.5, 0.5]`

In [6]:
## Modelos que debe ajustar

SVC_model = SVC(C=15, kernel='poly', degree=2)
RF_model = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=123, max_samples=0.8, max_features='log2')
NB_model = BernoulliNB(class_prior=[0.5, 0.5])

# stratified k fold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)

# funcion de cross vaalidation
def cv(X, y):

    recall_svc = cross_val_score(SVC_model, X, y, cv=skf, scoring='recall')
    recall_rf = cross_val_score(RF_model, X, y, cv=skf, scoring='recall')
    recall_gnb = cross_val_score(NB_model, X, y, cv=skf, scoring='recall')

    precision_svc = cross_val_score(SVC_model, X, y, cv=skf, scoring=metrics.make_scorer(metrics.precision_score, zero_division=0))

    precision_rf = cross_val_score(RF_model, X, y, cv=skf, scoring=metrics.make_scorer(metrics.precision_score, zero_division=0))
    precision_gnb = cross_val_score(NB_model, X, y, cv=skf, scoring=metrics.make_scorer(metrics.precision_score, zero_division=0))

    accuracy_svc = cross_val_score(SVC_model, X, y, cv=skf, scoring='accuracy')
    accuracy_rf = cross_val_score(RF_model, X, y, cv=skf, scoring='accuracy')
    accuracy_gnb = cross_val_score(NB_model, X, y, cv=skf, scoring='accuracy')

    f1_svc = cross_val_score(SVC_model, X, y, cv=skf, scoring='f1')
    f1_rf = cross_val_score(RF_model, X, y, cv=skf, scoring='f1')
    f1_gnb = cross_val_score(NB_model, X, y, cv=skf, scoring='f1')

    auc_svc = cross_val_score(SVC_model, X, y, cv=skf, scoring='roc_auc')
    auc_rf = cross_val_score(RF_model, X, y, cv=skf, scoring='roc_auc')
    auc_gnb = cross_val_score(NB_model, X, y, cv=skf, scoring='roc_auc')

    cv_results = pd.DataFrame(
        index=['SV', 'RF', 'NB'],
        columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])

    cv_results.loc['SV'] = [
        accuracy_svc.mean(),
        precision_svc.mean(),
        recall_svc.mean(),
        f1_svc.mean(),
        auc_svc.mean()
    ]
    cv_results.loc['RF'] = [
        accuracy_rf.mean(),
        precision_rf.mean(),
        recall_rf.mean(),
        f1_rf.mean(),
        auc_rf.mean()
    ]
    cv_results.loc['NB'] = [
        accuracy_gnb.mean(),
        precision_gnb.mean(),
        recall_gnb.mean(),
        f1_gnb.mean(),
        auc_gnb.mean()
    ]

    return cv_results

cv_results = cv(X_train, y_train)
cv_results

,Accuracy,Precision,Recall,F1,AUC
SV,0.957215,0.0,0.0,0.0,0.735307
RF,0.959747,0.3,0.133333,0.18,0.855702
NB,0.838829,0.051496,0.183333,0.078941,0.619408


In [7]:
# Test 1 - P3: Se comparan los valores de la matriz cv_results completa.

4. De acuerdo con los resultados anteriores, ¿cuál modelo recomedaría?. 

Para ello primero debe indicar la medida de desempeño en `medida`, donde debe escoger entre 'Accuracy', 'Precision', 'F1', 'Recall' y 'AUC'. (Por ejemplo, `medida='Accuracy'`). Una vez seleccionada la medida debe indicar el modelo de mejor desempeño, de acuerdo con la validación cruzada, en el objeto `model_select`, para indicar el mejor modelo utilizar `SV`, `RF` o `NB` (Por ejemplo, `model_select='SV'`).

In [8]:
medida = None            # Variable que debe modificar
model_select = None      # Variable que debe modificar

# your code here
medida = 'AUC'
model_select = 'RF'

In [9]:
# Test 1 - P4: Elección adecuada del modelo

5. También se debe evaluar el desempeño de los modelos, es decir, su capacidad predictiva o de generalización. Tener en cuenta, que por lo general, se debe evaluar solo la generalización del modelo escogido con la muestra de entrenamiento, cuando este último se considera aceptable. Solo con fines comparativos se evaluará la generalización de los 3 modelos propuestos.


Para esto, ajuste los modelos del punto anterior con los datos de entrenamiento y evalúelos con los datos de validación. Debe entregar un dataframe de nombre `test_results` con la siguiente estructura:

| |Accuracy|Precision|Recall|F1|AUC|
|-|--------|---------|------|--|-------|
|SV|-|-|-|-|-|
|RF|-|-|-|-|-|
|NB|-|-|-|-|-|

En python:
    
```python
# Dataframe vacío
test_results = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])
```

Considere los mismos hiperparámetros del punto anterior. En caso de división por cero se debe deja el indicar (o métrica) en 0, para ello utilice el argumente `zero_division=0`.

In [35]:
# your code here


def test_set(X,y):
    SVC_model.fit(X, y)
    RF_model.fit(X, y)
    NB_model.fit(X, y)

    y_pred_svc = SVC_model.predict(X_test)
    y_pred_rf = RF_model.predict(X_test)
    y_pred_gnb = NB_model.predict(X_test)

    accuracy_svc = metrics.accuracy_score(y_test, y_pred_svc)
    accuracy_rf = metrics.accuracy_score(y_test, y_pred_rf)
    accuracy_gnb = metrics.accuracy_score(y_test, y_pred_gnb)

    precision_svc = metrics.precision_score(y_test, y_pred_svc, zero_division=0)
    precision_rf = metrics.precision_score(y_test, y_pred_rf, zero_division=0)
    precision_gnb = metrics.precision_score(y_test, y_pred_gnb, zero_division=0)

    recall_svc = metrics.recall_score(y_test, y_pred_svc,zero_division=0)
    recall_rf = metrics.recall_score(y_test, y_pred_rf, zero_division=0)
    recall_gnb = metrics.recall_score(y_test, y_pred_gnb, zero_division=0)

    f1_svc = metrics.f1_score(y_test, y_pred_svc, zero_division=0)
    f1_rf = metrics.f1_score(y_test, y_pred_rf, zero_division=0)
    f1_gnb = metrics.f1_score(y_test, y_pred_gnb, zero_division=0)
    
    auc_svc = metrics.roc_auc_score(y_test, y_pred_svc)
    auc_rf = metrics.roc_auc_score(y_test, y_pred_rf)
    auc_gnb = metrics.roc_auc_score(y_test, y_pred_gnb)
    
    test_results = pd.DataFrame(index=['SV', 'RF', 'NB'], columns=['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'])
    test_results.loc['SV'] = [accuracy_svc, precision_svc, recall_svc, f1_svc, auc_svc]
    test_results.loc['RF'] = [accuracy_rf, precision_rf, recall_rf, f1_rf, auc_rf]
    test_results.loc['NB'] = [accuracy_gnb, precision_gnb, recall_gnb, f1_gnb, auc_gnb]

    return test_results
    

test_results = test_set(X_train, y_train)
test_results

,Accuracy,Precision,Recall,F1,AUC
SV,0.959064,0.0,0.0,0.0,0.5
RF,0.959064,0.0,0.0,0.0,0.5
NB,0.853801,0.090909,0.285714,0.137931,0.581882


In [34]:
# Test 1 - P5: Se comparan los valores de la matriz test_results completa.

Hasta ahora, hemos trabajado con los datos "originales", es decir, sin balancear las clases. Lo que produce este problema es la inclinación de los modelos a prever la clase mayoritaria, resultando en un rendimiento deficiente. Para solucionar esto, vamos a utilizar las técnicas de remuestreo mencionadas en la introducción del laboratorio.

Además, habrá notado que la métricas: `precision` y `recall` no han tenido un buen desempeño en los modelos anteriores. Esto se debe a que estas métricas están relacionadas con la clase minoritaria. Luego, para mejorar el desempeño de estas métricas, debemos balancear las clases.

Las siguientes preguntas tienen como propósito evaluar el rendimiento de los modelos previos utilizando datos balanceados. **Aunque es posible que cada modelo mejore con ajustes más precisos, el enfoque de este laboratorio es analizar el comportamiento de los modelos al equilibrar las clases con las diferentes técnicas disponibles**.

**MUY IMPORTANTE**: para las preguntas 6, 7 y 8 debe entregar los dataframes con el mismo formato anterior, es decir:

| |Accuracy|Precision|Recall|F1|AUC|
|-|--------|---------|------|--|-------|
|SV|-|-|-|-|-|
|RF|-|-|-|-|-|
|NB|-|-|-|-|-|

6 - Realice un sobremuestreo de la clase minoritaria utilizando la técnica `RandomOverSampler` y evalúe el desempeño de los modelos con los datos balanceados. Debe replicar lo solicitado en el punto 3 y 5, pero ahora con los datos balanceados. Debe entregar los siguientes objetos:

- `cv_results_balanced`: Dataframe con los resultados de la validación cruzada estratificada con 5 folds.
- `test_results_balanced`: Dataframe con los resultados de la evaluación de los modelos con los datos de validación.

Considere utilizar los siguientes hiperparámetros: `random_state=123` y `sampling_strategy='minority'` en `RandomOverSampler`.

In [12]:
# your code here
# over_sampling
oversample = RandomOverSampler(sampling_strategy='minority', random_state=123)

X_train_ros, y_train_ros = oversample.fit_resample(X_train, y_train)

cv_results_balanced = cv(X_train_ros, y_train_ros)

test_results_balanced = test_set(X_train_ros, y_train_ros)

In [13]:
# Test 1 - P6: Se comparan los valores de la matriz cv_results_balanced completa.

In [14]:
# Test 2 - P6: Se comparan los valores de la matriz test_results_balanced completa.

7 - Nuevamente realice la validación cruzada y la evaluación del conjunto de prueba para los modelos de clasificación, pero esta vez utilizando la técnica `RandomUnderSampler` para balancear las clases. Debe entregar los siguientes objetos:

- `cv_results_balanced2`: Dataframe con los resultados de la validación cruzada estratificada con 5 folds.
- `test_results_balanced2`: Dataframe con los resultados de la evaluación de los modelos con los datos de validación.

Considere utilizar los siguientes hiperparámetros: `random_state=123` y `sampling_strategy='majority'` en `RandomUnderSampler`.

In [15]:
# your code here
# under_sampling
undersample = RandomUnderSampler(sampling_strategy='majority', random_state=123)

X_train_rus, y_train_rus = undersample.fit_resample(X_train, y_train)

cv_results_balanced2 = cv(X_train_rus, y_train_rus)

test_results_balanced2 = test_set(X_train_rus, y_train_rus)

In [16]:
# Test 1 - P7: Se comparan los valores de la matriz cv_results_balanced2 completa.

In [17]:
# Test 2 - P7: Se comparan los valores de la matriz test_results_balanced2 completa.

8 - Por último, nuevamente balancear las clases, pero esta vez utilizando las siguientes técnicas de balanceo de clases:
- `NearMiss`
- `ADASYN`
- Combinación de `RandomOverSampler` y `RandomUnderSampler` con `sampling_strategy=0.5` en ambos.

Los nombres de los objetos deben ser los siguientes:
- `cv_results_balanced_nm`: Dataframe con los resultados de la validación cruzada estratificada con 5 folds utilizando `NearMiss`.
- `test_results_balanced_nm`: Dataframe con los resultados de la evaluación de los modelos con los datos de validación utilizando `NearMiss`.
- `cv_results_balanced_ad`: Dataframe con los resultados de la validación cruzada estratificada con 5 folds utilizando `ADASYN`.
- `test_results_balanced_ad`: Dataframe con los resultados de la evaluación de los modelos con los datos de validación utilizando `ADASYN`.
- `cv_results_balanced_comb`: Dataframe con los resultados de la validación cruzada estratificada con 5 folds utilizando `RandomOverSampler` y `RandomUnderSampler`.
- `test_results_balanced_comb`: Dataframe con los resultados de la evaluación de los modelos con los datos de validación utilizando `RandomOverSampler` y `RandomUnderSampler`.

Debe utilizar los siguientes hiperparámetros para cada técnica de balanceo:
- `NearMiss`: `sampling_strategy='majority'`
- `ADASYN`: `random_state=123`, `sampling_strategy='minority'`
- `RandomOverSampler` y `RandomUnderSampler`: `random_state=123`, `sampling_strategy=0.5`

In [18]:
# your code here
# NearMiss
nm = NearMiss(sampling_strategy='majority')
X_train_nm, y_train_nm = nm.fit_resample(X_train, y_train)
cv_results_balanced_nm = cv(X_train_nm, y_train_nm)
test_results_balanced_nm = test_set(X_train_nm, y_train_nm)

# ADASYN
ad = ADASYN(random_state=123, sampling_strategy='minority')
X_train_ad, y_train_ad = ad.fit_resample(X_train, y_train)
cv_results_balanced_ad = cv(X_train_ad, y_train_ad)
test_results_balanced_ad = test_set(X_train_ad, y_train_ad)

# Combinación de RandomOverSampler y RandomUnderSampler
ros = RandomOverSampler(random_state=123, sampling_strategy=0.5)
rus = RandomUnderSampler(random_state=123, sampling_strategy=0.5)
X_train_comb, y_train_comb = ros.fit_resample(X_train, y_train)
X_train_comb, y_train_comb = rus.fit_resample(X_train_comb, y_train_comb)
cv_results_balanced_comb = cv(X_train_comb, y_train_comb)
test_results_balanced_comb = test_set(X_train_comb, y_train_comb)

In [19]:
# Test 1 - P8: NearMiss (se comparan los valores de las matrices cv_results_balanced_nm y test_results_balanced_nm)

In [20]:
# Test 2 - P8: ADASYN (se comparan los valores de las matrices cv_results_balanced_ad y test_results_balanced_ad)

In [21]:
# Test 3 - P8: Combinación de técnicas oversampling y undersampling (se comparan los valores de las matrices cv_results_balanced_comb y test_results_balanced_comb)

9. Considerando los resultados obtenidos desde el punto 6 hasta el punto 8 (técnicas de balance de datos), ¿cuál es la mejor técnica de balanceo de datos para este problema? Para responder esta pregunta considere solamente la métrica de desempeño `F1-Score` en el dataset de entrenamiento. 

El nombre del modelo dede ser `best_model_balancing` con una de las siguientes opciones: 'SV', 'RF', 'NB'. Por ejemplo:`best_model_balancing='SV'`.

Además, debe entregar un objeto de tipo `int` con el nombre `best_balancing` que contenga la codificación correspondiente a la mejor técnica de balanceo de datos según la siguiente tabla:

|Técnica|Código|
|-------|------|
|RandomOverSampler|1|
|RandomUnderSampler|2|
|NearMiss|3|
|ADASYN|4|
|Combinación de RandomOverSampler y RandomUnderSampler|5|

Por ejemplo: `best_balancing=1`

In [22]:
best_model_balancing = None    # Objeto que debe modificar
best_balancing = None          # Objeto que debe modificar

# your code here
# Extraer el mejor F1-score de cada técnica y modelo
f1_ros = cv_results_balanced['F1']
f1_rus = cv_results_balanced2['F1']
f1_nm = cv_results_balanced_nm['F1']
f1_ad = cv_results_balanced_ad['F1']
f1_comb = cv_results_balanced_comb['F1']

f1_ros = f1_ros.astype(float)
f1_rus = f1_rus.astype(float)
f1_nm = f1_nm.astype(float)
f1_ad = f1_ad.astype(float)
f1_comb = f1_comb.astype(float)


# Guardar los valores máximos y sus índices
max_f1_ros = f1_ros.max()
model_ros = f1_ros.idxmax()
max_f1_rus = f1_rus.max()
model_rus = f1_rus.idxmax()
max_f1_nm = f1_nm.max()
model_nm = f1_nm.idxmax()
max_f1_ad = f1_ad.max()
model_ad = f1_ad.idxmax()
max_f1_comb = f1_comb.max()
model_comb = f1_comb.idxmax()

# Comparar los máximos entre técnicas
f1_scores = [max_f1_ros, max_f1_rus, max_f1_nm, max_f1_ad, max_f1_comb]

models = [model_ros, model_rus, model_nm, model_ad, model_comb]
balancing_codes = [1, 2, 3, 4, 5]
best_idx = int(np.argmax(f1_scores))
best_model_balancing = models[best_idx]
best_balancing = balancing_codes[best_idx]

print(f"Mejor técnica de balanceo: código {best_balancing}")
print(f"Mejor modelo: {best_model_balancing}")


Mejor técnica de balanceo: código 1
Mejor modelo: RF


In [23]:
# Test 1 - P9

10. Considerando los resultados obtenidos desde el punto 6 hasta el punto 8 (técnicas de balance de datos), ¿cuál es la mejor técnica de balanceo de datos para este problema? Para responder esta pregunta considere solamente la métrica de desempeño `F1-Score` en el dataset de test. (Esta comparación es solo confines de comprender la diferencia entre los errores cometidos en la muestra train y test, no se debe seleccionar el modelo desde la muestra test) 

El nombre del modelo dede ser `best_model_balancing_tets` con una de las siguientes opciones: 'SV', 'RF', 'NB'. Por ejemplo:`best_model_balancing_test='SV'`.

Además, debe entregar un objeto de tipo `int` con el nombre `best_balancing_test` que contenga la codificación correspondiente a la mejor técnica de balanceo de datos según la siguiente tabla:

|Técnica|Código|
|-------|------|
|RandomOverSampler|1|
|RandomUnderSampler|2|
|NearMiss|3|
|ADASYN|4|
|Combinación de RandomOverSampler y RandomUnderSampler|5|

Por ejemplo: `best_balancing_test=1`

In [24]:
best_model_balancing_test = None    # Objeto que debe modificar
best_balancing_test = None          # Objeto que debe modificar

# your code here
# Extraer el mejor F1-score de cada técnica y modelo en el dataset de test
f1_ros_test = test_results_balanced['F1'].astype(float)
f1_rus_test = test_results_balanced2['F1'].astype(float)
f1_nm_test = test_results_balanced_nm['F1'].astype(float)
f1_ad_test = test_results_balanced_ad['F1'].astype(float)
f1_comb_test = test_results_balanced_comb['F1'].astype(float)

max_f1_ros_test = f1_ros_test.max()
model_ros_test = f1_ros_test.idxmax()
max_f1_rus_test = f1_rus_test.max()
model_rus_test = f1_rus_test.idxmax()
max_f1_nm_test = f1_nm_test.max()
model_nm_test = f1_nm_test.idxmax()
max_f1_ad_test = f1_ad_test.max()
model_ad_test = f1_ad_test.idxmax()
max_f1_comb_test = f1_comb_test.max()
model_comb_test = f1_comb_test.idxmax()

f1_scores_test = [max_f1_ros_test, max_f1_rus_test, max_f1_nm_test, max_f1_ad_test, max_f1_comb_test]
models_test = [model_ros_test, model_rus_test, model_nm_test, model_ad_test, model_comb_test]
balancing_codes_test = [1, 2, 3, 4, 5]
best_idx_test = int(np.argmax(f1_scores_test))
best_model_balancing_test = models_test[best_idx_test]
best_balancing_test = balancing_codes_test[best_idx_test]

In [25]:
# Test 1 - P10

11. Por último, responda las siguientes preguntas, en ambas considere como métrica el `F1-Score`:

a. El mejor método (modelo de clasificación y técnica de balanceo) con la muestra de entrenamiento coincide con el mejor de la muestra test. Para esto, el objeto `resp_a` debe contener algunas de las siguientes:
- 'Los métodos coinciden'
- 'Los métodos coinciden solo en el modelo pero no en la técnica de balanceo'
- 'Los métodos coinciden solo en la técnica de balanceo pero no en el modelo'
- 'Los métodos no coinciden'

Por ejemplo: `resp_a='Los métodos coinciden'`

b. ¿Cuál de las dos métodologias, utilizando los datos originales o utilizando los datos balanceados, tiene mejor generalización en la muestra test. Para esto, el objeto `resp_b` debe contener algunas de las siguientes:
- 'Los métodos tienen prácticamente la misma generalización, es decir, la diferencia es menor a un 1%'
- 'Utilizando los datos originales tiene mejores resultados'
- 'Utilizando los datos balanceados tiene mejores resultados'

Por ejemplo: `resp_b='Los métodos tienen prácticamente la misma generalización, es decir, la diferencia es menor a un 1%'`

In [27]:
resp_a = None                         # Objeto que debe modificar
resp_b = None                         # Objeto que debe modificar

# your code here
resp_a = 'Los métodos coinciden solo en el modelo pero no en la técnica de balanceo'
resp_b = 'Utilizando los datos balanceados tiene mejores resultados'

In [28]:
# Test 1 - P11